In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
59,application_1607949680860_0061,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import hashlib
from datetime import datetime
from graphframes import *
from pyspark.sql import functions as func
from pyspark.sql.types import FloatType
import hsfs

import os
from pyspark.sql import SQLContext

In [3]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [13]:
edge_fg = fs.get_feature_group('transactions_fg', 1)
node_fg = fs.get_feature_group('account_features', 1)

In [14]:
node_fg.show(5)

+-------+---------------+--------------+---------+
|acct_id|initial_deposit|tx_behavior_id|prior_sar|
+-------+---------------+--------------+---------+
|      0|       84442.19|             1|        0|
|      1|       75795.44|             1|        0|
|      2|       42057.16|             1|        0|
|      3|       25891.68|             1|        0|
|      4|       51127.47|             1|        0|
+-------+---------------+--------------+---------+
only showing top 5 rows

In [15]:
edge_fg.show(5)

+------+-------+--------------+--------+------+--------+------+-------+
|is_sar|tran_id|tran_timestamp|base_amt|target|alert_id|source|tx_type|
+------+-------+--------------+--------+------+--------+------+-------+
|     0|      1|   1.4832288E9| 9405.71|  3259|      -1|  1767|      4|
|     0|      2|   1.4832288E9| 6884.54|  5141|      -1|  7363|      4|
|     0|      3|   1.4832288E9|  7968.4|  9532|      -1|  7585|      4|
|     0|      4|   1.4832288E9| 9042.67|  8792|      -1|  1750|      4|
|     0|      5|   1.4832288E9| 4692.79|  4670|      -1|  9060|      4|
+------+-------+--------------+--------+------+--------+------+-------+
only showing top 5 rows

In [17]:
only_sar_edge_df = edge_fg.read().filter(func.col('alert_id') != -1)
only_normal_edge_df =  edge_fg.read().filter(func.col('alert_id') == -1)

In [22]:
only_normal_edge_df.show(5)

+------+-------+--------------+--------+------+--------+------+-------+
|is_sar|tran_id|tran_timestamp|base_amt|target|alert_id|source|tx_type|
+------+-------+--------------+--------+------+--------+------+-------+
|     0|      1|   1.4832288E9| 9405.71|  3259|      -1|  1767|      4|
|     0|      2|   1.4832288E9| 6884.54|  5141|      -1|  7363|      4|
|     0|      3|   1.4832288E9|  7968.4|  9532|      -1|  7585|      4|
|     0|      4|   1.4832288E9| 9042.67|  8792|      -1|  1750|      4|
|     0|      5|   1.4832288E9| 4692.79|  4670|      -1|  9060|      4|
+------+-------+--------------+--------+------+--------+------+-------+
only showing top 5 rows

In [23]:
only_sar_edge_df.show()

+------+-------+--------------+--------+------+--------+------+-------+
|is_sar|tran_id|tran_timestamp|base_amt|target|alert_id|source|tx_type|
+------+-------+--------------+--------+------+--------+------+-------+
|     1|    439|   1.4832288E9|  194.53|  7950|       0|  8485|      4|
|     1| 100277|   1.4892768E9|  194.53|  7950|       0|  2322|      4|
|     1|  20677|   1.4844384E9|  194.53|  7950|       0|  3838|      4|
|     1| 120778|   1.4904864E9|  194.53|  7950|       0|  3948|      4|
|     1| 141265|    1.491696E9|  194.53|  7950|       0|  6419|      4|
|     1|  60292|   1.4868576E9|  194.53|  7950|       0|  8913|      4|
|     1| 161186|   1.4929056E9|  194.53|  7950|       0|  7133|      4|
|     1|  40370|    1.485648E9|  194.53|  7950|       0|  2145|      4|
|     1|  80897|   1.4880672E9|  194.53|  7950|       0|  6075|      4|
|     1| 105719|    1.489536E9|  189.75|  5470|       1|   511|      4|
|     1| 210181|   1.4958432E9|  189.75|  5470|       1|  7236| 

In [18]:
only_sar_edge_df.count()

732

In [19]:
only_normal_edge_df.count()

1028964

In [25]:
only_sar_edge_df = only_sar_edge_df.sort('alert_id')
only_sar_edge_df_grouped = only_sar_edge_df.groupBy('alert_id','tran_id').agg(func.min("tran_timestamp"),func.max("tran_timestamp")).toDF("alert_id","tran_id", "window_start", "window_end")
only_sar_edge_df_grouped.show(5)

+--------+-------+------------+-----------+
|alert_id|tran_id|window_start| window_end|
+--------+-------+------------+-----------+
|       0|    439| 1.4832288E9|1.4832288E9|
|       0|  20677| 1.4844384E9|1.4844384E9|
|       0|  40370|  1.485648E9| 1.485648E9|
|       0|  60292| 1.4868576E9|1.4868576E9|
|       0|  80897| 1.4880672E9|1.4880672E9|
+--------+-------+------------+-----------+
only showing top 5 rows

In [27]:
only_normal_edge_df_grouped = only_sar_edge_df_grouped.select("window_start", "window_end").join(
    only_normal_edge_df,
    [(only_normal_edge_df.tran_timestamp>=only_sar_edge_df_grouped.window_start)&(only_normal_edge_df.tran_timestamp<=only_sar_edge_df_grouped.window_end)],
    how="left"
)

In [28]:
only_normal_edge_df_grouped.show()

+------------+-----------+------+-------+--------------+--------+------+--------+------+-------+
|window_start| window_end|is_sar|tran_id|tran_timestamp|base_amt|target|alert_id|source|tx_type|
+------------+-----------+------+-------+--------------+--------+------+--------+------+-------+
| 1.4832288E9|1.4832288E9|     0|      1|   1.4832288E9| 9405.71|  3259|      -1|  1767|      4|
| 1.4832288E9|1.4832288E9|     0|      2|   1.4832288E9| 6884.54|  5141|      -1|  7363|      4|
| 1.4832288E9|1.4832288E9|     0|      3|   1.4832288E9|  7968.4|  9532|      -1|  7585|      4|
| 1.4832288E9|1.4832288E9|     0|      4|   1.4832288E9| 9042.67|  8792|      -1|  1750|      4|
| 1.4832288E9|1.4832288E9|     0|      5|   1.4832288E9| 4692.79|  4670|      -1|  9060|      4|
| 1.4832288E9|1.4832288E9|     0|      6|   1.4832288E9| 4089.65|  3861|      -1|  8752|      4|
| 1.4832288E9|1.4832288E9|     0|      7|   1.4832288E9| 3055.04|  3805|      -1|  9645|      4|
| 1.4832288E9|1.4832288E9|    

In [29]:
only_normal_edge_df_grouped.count()

1048084

In [7]:
# iterates over the different alerts and collect the normal transactions in the time interval
for row in only_alert_edge_df_grouped.rdd.collect():
    min_ts = row['min(tran_timestamp)']
    max_ts = row['max(tran_timestamp)']
    alert_id = row['alert_id']
    
    tmp_df = only_normal_edge_df.filter((func.col('tran_timestamp') >= min_ts) & (func.col('tran_timestamp') <= max_ts))              
    g = GraphFrame(node_df, tmp_df)
    sc.setCheckpointDir("hdfs:///Projects/AML/Logs/sc")
    cc = g.connectedComponents()
    #cc.save.parquet("hdfs:///Projects/AML/Logs/parquet")
    cc_fg_meta = fs.create_feature_group(name="connected_components_alert_id_"+str(alert_id),
                                       version=1,
                                       primary_key=["id"],
                                       description="connected components of normal transactions within the "+str(alert_id) + " time interval.",
                                       time_travel_format=None,                                        
                                       statistics_config=False)
    cc_fg_meta.save(cc)

In [8]:
result =  spark.read.parquet("hdfs:///Projects/AML/Logs/sc/6e297fee-a4d4-4e0d-96a9-b2bf1dd0ba03")
display(result)

An error was encountered:
'Unable to infer schema for Parquet. It must be specified manually.;'
Traceback (most recent call last):
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 316, in parquet
    return self._df(self._jreader.parquet(_to_seq(self._spark._sc, paths)))
  File "/srv/hops/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Unable to infer schema for Parquet. It must be specified manually.;'

